# Glaciers Distribution

## Import packages

In [ ]:
import fsspec
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from c3s_eqc_automatic_quality_control import download

plt.style.use("seaborn-v0_8-notebook")

## Define request

In [ ]:
collection_id = "insitu-glaciers-extent"
request = {
    "variable": "all",
    "format": "zip",
    "version": "6_0",
}

## Download and open data

In [ ]:
# Takes a couple of minutes
ds = download.download_and_transform(collection_id, request)
gdf = gpd.GeoDataFrame(
    ds.to_pandas(),
    geometry=gpd.points_from_xy(ds["CENLON"], ds["CENLAT"]),
    crs="EPSG:4326",
)

## Print some statistics

In [ ]:
n_glaciers = len(gdf)
total_area = gdf["AREA"].sum()
print(
    " ".join(
        [
            f"A total number of {n_glaciers} glaciers is present in the dataset,",
            f"covering a total area of {total_area:.2f} km^2.",
        ]
    )
)

## Plotting function

In [ ]:
def plot_map(gdf, var_name=None, label=None, title=None, **kwargs):
    kwargs = {"markersize": 5, "legend": var_name is not None} | kwargs
    if var_name:
        kwargs = {"c": var_name, "column": var_name} | kwargs
        kwargs.setdefault("legend_kwds", {"shrink": 0.49, "extend": "both"})
        if label is not None:
            kwargs["legend_kwds"].setdefault("label", label)
    shapefile_url = "https://naturalearth.s3.amazonaws.com/110m_cultural/ne_110m_admin_0_countries.zip"
    with fsspec.open(f"simplecache::{shapefile_url}") as f:
        gdf_countries = gpd.read_file(f)

    ax = gdf_countries.boundary.plot(
        figsize=(20, 16), facecolor="none", edgecolor="black", linewidth=0.25
    )
    gdf.plot(ax=ax, **kwargs)
    ax.axis("off")
    if title:
        ax.set_title(title, fontsize=25)
    return ax

## Plot glaciers location

In [ ]:
ax = plot_map(
    gdf,
    color="red",
    title="Glacier distribution around the year 2000 according to the RGI v6.0",
)

## Plot glaciers distribution over years: Bars

In [ ]:
year = pd.DataFrame(
    [gdf["BGNDATE"].str[:4].astype(int), gdf["ENDDATE"].str[:4].astype(int)]
)
gdf["year"] = year.where(year > 0).mean().round().astype("Int64")
size = gdf.set_index("year").groupby("year").size()
ax = size.plot.bar(figsize=(15, 5), grid=True, ylabel="Number of glaciers")

## Plot glaciers distribution over years: Map

In [ ]:
cmap = mpl.cm.turbo
norm = mpl.colors.BoundaryNorm(range(1940, 2021, 10), cmap.N)
ax = plot_map(
    gdf,
    var_name="year",
    cmap=cmap,
    norm=norm,
    label="year",
    title="Glacier distribution around the year 2000 according to the RGI v6.0",
)